In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv('music.csv')

df.info()

df.head()

df.isnull().sum()  

df = df.drop(columns=["Unnamed: 0"])

df["release_date"] = pd.to_datetime(df["release_date"], format="%Y")

duplicates = df[df.duplicated(subset=["artist_name", "track_name"])]
print("Duplicatas:", len(duplicates))

df["genre"] = df["genre"].str.lower().str.strip()
df["topic"] = df["topic"].str.lower().str.strip()

print(df.describe())

print(df.groupby("genre")[["danceability", "energy", "valence"]].mean())


df["decade"] = (df["release_date"].dt.year // 10) * 10
print(df.groupby("decade")[["danceability", "energy", "valence"]].mean())

songs_decade = df["decade"].value_counts().sort_index()
top_genres = df["genre"].value_counts().head(10)
top_artists = df["artist_name"].value_counts().head(10)
top_topics = df["topic"].value_counts().head(10)
print(songs_decade, top_genres, top_artists, top_topics)

corr = df[["danceability","energy","valence","loudness","acousticness","instrumentalness"]].corr()
print(corr)